In [7]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sk
import pickle
import mne
from tqdm import trange

import torch
import torch_directml
from torch import nn

In [8]:
data=np.load('/home/pigmaster96/openneuroNMAdata/ds005540-download/derivatives/sub-01/ses-vid/eeg/sub-01_ses-vid_task-emotion_reorder.npy',allow_pickle=True)
data=np.permute_dims(data,[1,0,2])
data.shape


(21, 64, 6000)

In [9]:
#feature conversion from preprocessed signal to power spectrum, welch method with default params
data=np.load('/home/pigmaster96/openneuroNMAdata/ds005540-download/derivatives/sub-01/ses-vid/eeg/sub-01_ses-vid_task-emotion_reorder.npy',allow_pickle=True)
data=np.permute_dims(data,[1,0,2])
data.shape

cnames=['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3','AF4','AF8', 'F7', 'F5','F3','F1','Fz', 'F2', 'F4', 'F6', 'F8',
'FT7', 'FC5', 'FC3', 'FC1','FCz','FC2','FC4', 'FC6', 'FT8', 'T7','C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
'TP7', 'CP5', 'CP3', 'CP1','CPz','CP2', 'CP4','CP6', 'TP8', 'P7','P5', 'P3', 'P1', 'Pz','P2', 'P4', 'P6', 'P8',
'PO7', 'PO3','POz', 'PO4','PO8', 'O1','Oz','O2', 'F9', 'F10', 'TP9', 'TP10']
info=mne.create_info(cnames,200,'eeg')

def eeg_to_spectrum(eeg,info):
    epochs=mne.EpochsArray(data,info)
    out=epochs.compute_psd(method='welch',fmin=0.1,fmax=47,verbose=None).get_data()
    return out

temp=eeg_to_spectrum(data,info=info)
temp.shape

Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 10.240 (s)


(21, 64, 480)

In [10]:
#feature conversion from preprocessed signal to power spectrum, welch method with default params
def eeg_to_spectrum(eeg,info):
    epochs=mne.EpochsArray(data,info)
    out=epochs.compute_psd(method='welch',fmin=0.1,fmax=47,verbose=None).get_data()
    return out


#now create input vector
#14/7/25: Kernel keeps crashing when iterating from subject 1 to 54 in one go,
#this doesn't happen when iterating from 1-25 or 30-54... maybe the vector is too large?
def extract_data(sub_range,verbose=False,feature_func=None,**kwargs):
    first=sub_range[0]
    last=sub_range[-1]

    for sub in trange(first,last+1): 
        #read data
        if sub <=9:
            path=f'/home/pigmaster96/openneuroNMAdata/ds005540-download/derivatives/sub-0{sub}/ses-vid/eeg/sub-0{sub}_ses-vid_task-emotion_reorder.npy'
        else:
            path=f'/home/pigmaster96/openneuroNMAdata/ds005540-download/derivatives/sub-{sub}/ses-vid/eeg/sub-{sub}_ses-vid_task-emotion_reorder.npy'
        datatemp=np.load(path,allow_pickle=True)
        datatemp=np.permute_dims(datatemp,[1,0,2])

        #if we want to convert features
        if not feature_func==None:
            datatemp=feature_func(datatemp,info=info)

        #cat data along new dimension
        datatemp=torch.tensor(datatemp).unsqueeze(0)
        if sub==first:
            data=datatemp
        else:
            data=torch.cat([data,datatemp],axis=0)

        if verbose:
            print(f'subject {sub} data extracted, vector size: {data.shape}')
    return data


#create labels
#sad-dis-fear-neu-joy-ten-ins correspond to 1-7 respectively, each sample has 21 trials, for (7 emotions x 3 trials)
labels=np.array([])
for i in range(1,8):
    for n in range(0,3):
        labels=np.concatenate([labels,np.array([i])],axis=0)

In [11]:
data_1=extract_data([1,25],feature_func=eeg_to_spectrum,info=info)

  0%|          | 0/25 [00:00<?, ?it/s]

Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 10.240 (s)


  4%|▍         | 1/25 [00:00<00:10,  2.35it/s]

Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 10.240 (s)


  8%|▊         | 2/25 [00:00<00:09,  2.31it/s]

...

 92%|█████████▏| 23/25 [00:10<00:00,  2.11it/s]

Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 10.240 (s)


 96%|█████████▌| 24/25 [00:11<00:00,  2.05it/s]

Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 10.240 (s)


100%|██████████| 25/25 [00:11<00:00,  2.17it/s]


In [12]:
print(data_1.shape)
print(labels.shape)

torch.Size([25, 21, 64, 480])
(21,)
